In [16]:
import numpy as np
from DDGelectron_driftdiffusion import *
from DDGnlpoisson import *
from func_lib import *
# from DDGgummelmap import *
# Define sample inputs
psi = np.linspace(0, 1, 10)  # Electric potential
xaxis = np.linspace(0, 1, 10)  # Integration domain
ng = np.ones(10)  # Initial guess and BCs for electron density
p = np.ones(10) * 1e15  # Hole density
ni = 1.45e10  # Intrinsic carrier concentration
TAUN0 = 1e-7  # Electron lifetime
TAUP0 = 1e-7  # Hole lifetime
mun = 1350  # Electron mobility
fi_e = np.zeros(10)  # Electric quasi-Fermi level for electrons
fi_h = np.zeros(10)  # Electric quasi-Fermi level for holes
model = type('Model', (object,), {"N_wells_virtual": 2})  # Model instance
Vt = 0.026  # Thermal voltage
idata = type('Idata', (object,), {"wfh_general": 0, "wfe_general": 0, "E_state_general": 0, 
                                  "E_statec_general": 0, "meff_state_general": 0, "meff_statec_general": 0, 
                                  "n": 0})  # Input data instance

# Call the function
n = DDGelectron_driftdiffusion(psi, xaxis, ng, p, ni, TAUN0, TAUP0, mun, fi_e, fi_h, model, Vt, idata)

# Output the result
print("Updated electron density:", n)


Updated electron density: [1.00000000e+00 1.61017958e+00 2.10248201e+05 2.10250000e+05
 2.10250000e+05 2.10250000e+05 2.10250000e+05 2.10248390e+05
 1.79940610e+00 1.00000000e+00]


In [17]:
import numpy as np
from DDGhole_driftdiffusion import *

error=1e-5
# Define sample inputs
psi = np.linspace(0, 1, 10)  # Electric potential
xaxis = np.linspace(0, 1, 10)  # Spatial grid
pg = np.ones(10)  # Initial guess and BCs for hole density
n = np.ones(10) * 1e15  # Electron density
ni = 1.45e10  # Intrinsic carrier concentration
TAUN0 = 1e-7  # Electron lifetime
TAUP0 = 1e-7  # Hole lifetime
mup = 450  # Hole mobility
fi_e = np.zeros(10)  # Electric quasi-Fermi level for electrons
fi_h = np.zeros(10)  # Electric quasi-Fermi level for holes
model = type('Model', (object,), {"N_wells_virtual": 2})  # Model instance
Vt = 0.026  # Thermal voltage
idata = type('Idata', (object,), {"wfh_general": 0, "wfe_general": 0, "E_state_general": 0, 
                                  "E_statec_general": 0, "meff_state_general": 0, "meff_statec_general": 0, 
                                  "p": 0})  # Input data instance

# Call the function
p = DDGhole_driftdiffusion(psi, xaxis, pg, n, ni, TAUN0, TAUP0, mup, fi_e, fi_h, model, Vt, idata)

# Output the result
print("Updated hole density:", p)

Updated hole density: [1.00000000e+00 1.79940610e+00 2.10248390e+05 2.10250000e+05
 2.10250000e+05 2.10250000e+05 2.10250000e+05 2.10248201e+05
 1.61017958e+00 1.00000000e+00]


In [18]:
def DDGgummelmap(n_max, xaxis, idata, odata, toll, maxit, ptoll, pmaxit, verbose, ni, fi_e, fi_h, model, Vt, error):
    odata  = idata
    dop = idata.dop
    Ppz_Psp = idata.Ppz_Psp
    vout = np.zeros((n_max, 2))
    hole_density = np.zeros((n_max, 3))
    electron_density = np.zeros((n_max, 3))
    fermin = np.zeros((n_max, 2))
    fermip = np.zeros((n_max, 2))
    v_Nnodes = np.arange(n_max)

    vout[:,0] = idata.V
    hole_density[:,0] = idata.p
    electron_density[:,0] = idata.n
    fermin[:,0] = idata.Fn
    fermip[:,0] = idata.Fp
    nrm = np.zeros(maxit)

    print("=======================================")
    for i in range(maxit):
        if verbose > 1:
            print("*****************************************************************")
            print("****    start of gummel iteration number:", i)
            print("*****************************************************************")
            print("solving non linear poisson equation")

        # Solve the non-linear Poisson equation
        [vout[:,1], electron_density[:,1], hole_density[:,1]] = DDGnlpoisson(
            idata, xaxis, v_Nnodes, vout[:,0], electron_density[:,0], hole_density[:,0], 
            fermin[:,0], fermip[:,0], dop, Ppz_Psp, idata.l2, ptoll, pmaxit, verbose, ni, fi_e, fi_h, model, Vt
        )

        if verbose > 1:
            print("updating electron qfl")

        # Update electron quasi-Fermi level
        electron_density[:,2] = DDGelectron_driftdiffusion(
            vout[:,1], xaxis, electron_density[:,1], hole_density[:,1], idata.nis, 
            idata.TAUN0, idata.TAUP0, idata.mun, fi_e, fi_h, model, Vt, idata
        )

        fermin[:,1] = DDGn2phin(vout[:,1], electron_density[:,2])
        fermin[0,1] = idata.Fn[0]
        fermin[n_max-1,1] = idata.Fn[-1]

        if verbose > 1:
            print("updating hole qfl")

        # Update hole quasi-Fermi level
        hole_density[:,2] = DDGhole_driftdiffusion(
            vout[:,1], xaxis, hole_density[:,1], electron_density[:,1], idata.nis, 
            idata.TAUN0, idata.TAUP0, idata.mup, fi_e, fi_h, model, Vt, idata
        )

        fermip[:,1] = DDGp2phip(vout[:,1], hole_density[:,2])
        fermip[0,1] = idata.Fp[0]
        fermip[n_max-1,1] = idata.Fp[-1]

        if verbose > 1:
            print("checking for convergence")

        nrfn = np.linalg.norm(fermin[:,1] - fermin[:,0], np.inf)
        nrfp = np.linalg.norm(fermip[:,1] - fermip[:,0], np.inf)
        nrv = np.linalg.norm(vout[:,1] - vout[:,0], np.inf)
        nrm[i] = max(nrfn, nrfp, nrv)

        if verbose > 1:
            print("max(|phin_(k+1)-phin_(k)| , |phip_(k+1)-phip_(k)| , |v_(k+1)- v_(k)|) =", nrm[i])

        if nrm[i] < error:
            break

        vout[:,0] = vout[:,1]
        hole_density[:,0] = hole_density[:,2]
        electron_density[:,0] = electron_density[:,2]
        fermin[:,0] = fermin[:,1]
        fermip[:,0] = fermip[:,1]

    it = i
    res = nrm

    if verbose > 0:
        print("\n\nInitial guess computed by DD: # of Gummel iterations =", it)

    odata.n = electron_density[:,2]
    odata.p = hole_density[:,2]
    odata.V = vout[:,1]
    odata.Fn = fermin[:,1]
    odata.Fp = fermip[:,1]

    return [odata, it, res]

In [19]:

# Assuming all necessary functions and configurations are correctly imported and defined
# Define the necessary imports and functions for your model

# Define sample inputs for DDGgummelmap
n_max = 1000
xaxis = np.linspace(0, 1, n_max)  # Spatial grid
ni = 1.45e10  # Intrinsic carrier concentration
fi_e = np.zeros(n_max)  # Electric quasi-Fermi level for electrons
fi_h = np.zeros(n_max)  # Electric quasi-Fermi level for holes
Vt = 0.026  # Thermal voltage

# Define your model and idata objects
model = type('Model', (object,), {"N_wells_virtual": 2, "quantum_effect": True})
idata = type('Idata', (object,), {
    "dop": np.ones(n_max),
    "Ppz_Psp": np.zeros(n_max),
    "l2": 1.0,
    "nis": ni,
    "TAUN0": 1e-7,
    "TAUP0": 1e-7,
    "mun": 1400,
    "mup": 450,
    "wfh_general": 0,
    "wfe_general": 0,
    "E_state_general": 0,
    "E_statec_general": 0,
    "meff_state_general": 0,
    "meff_statec_general": 0,
    "p": np.ones(n_max) * 1e15,
    "n": np.ones(n_max) * 1e15,
    "Fn": np.zeros(n_max),
    "Fp": np.zeros(n_max),
    "V": np.zeros(n_max)
})

odata = idata  # Assuming initial guess is same as idata for simplicity
toll = 1e-5
maxit = 1000
ptoll = 1e-5
error=1e-5
pmaxit = 1000
error=1e-5
verbose = 2

In [20]:
# Call the DDGgummelmap function
result_odata, iterations, residuals = DDGgummelmap(n_max=n_max, 
                                                   xaxis=xaxis, 
                                                   idata=idata, 
                                                   odata=odata, 
                                                   toll=toll, 
                                                   maxit=maxit, 
                                                   ptoll=ptoll, 
                                                   pmaxit=pmaxit, 
                                                   verbose=verbose, 
                                                   ni=ni, 
                                                   fi_e=fi_e, 
                                                   fi_h=fi_h, 
                                                   model=model, 
                                                   Vt=Vt, 
                                                   error=error)

# Output the results
print("Resulting odata:", result_odata)
print("Number of iterations:", iterations)
print("Residuals:", residuals)

*****************************************************************
****    start of gummel iteration number: 0
*****************************************************************
solving non linear poisson equation

 newton iteration: 1, reldVnorm = 1.000000

 damping iteration: 1, residual norm = 0.001001

 damping iteration: 2, residual norm = 0.430714

 damping iteration: 3, residual norm = 0.086944

 damping iteration: 4, residual norm = 0.018190

 damping iteration: 5, residual norm = 0.004439

 damping iteration: 6, residual norm = 0.001689

 damping iteration: 7, residual norm = 0.001139

 damping iteration: 8, residual norm = 0.001029

 damping iteration: 9, residual norm = 0.001007

 newton iteration: 2, reldVnorm = 0.200000

 damping iteration: 1, residual norm = 0.001002

exiting damping cycle because residual norm = 0.000000 


 newton iteration: 3, reldVnorm = 0.999997

 damping iteration: 1, residual norm = 0.000000

exiting damping cycle because residual norm = 0.000000 


